In [ ]:
# Point Colaboratory to your Google Drive

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
datapath = '/content/gdrive/My Drive/^DATA SCIENCE/^Biological Data/PROJECT/models/' # Luca
# datapath = '/content/gdrive/My Drive/Data Science - Unipd/II Year/I Semester/Biological Data/Project/PROJECT/models/' # Giuliano

# Building models

## Ex 5: build a PSSM model

In [ ]:
# ./binx/ncbi-blast-2.11.0+/bin/psiblast -in_msa ./PROJECT/clustal_omega_uniref50.fasta -db ./data/msa/uniprot_sprot.fasta -out ./PROJECT/output_psiblast_uniref50.out -out_pssm ./PROJECT/pssm_model_uniref50.pssm

## Ex 6: build a HMM model

In [ ]:
# ./binx/hmmer-3.3.1/src/hmmbuild ./PROJECT/hmm_model_uniref50.hmm ./PROJECT/clustal_omega_uniref50.fasta

## Ex 7: find significant hits using HMM-SEARCH and PSI-BLAST

In [ ]:
# hmmsearch --tblout ./PROJECT/hmm_search_uniref50.hmmer_tblout --domtblout ./PROJECT/hmm_search_uniref50.hmmer_domtblout ./PROJECT/hmm_model_uniref50.hmm ./data/msa/uniprot_sprot.fasta > ./PROJECT/hmm_search_uniref50.hmmer_align

In [ ]:
# ./binx/ncbi-blast-2.11.0+/bin/psiblast -in_pssm ./PROJECT/pssm_model_uniref50.pssm -db ./data/msa/uniprot_sprot.fasta -outfmt 6 -num_iterations 4 -evalue 0.01 > ./PROJECT/psiblast_search_uniref50.txt

## Ex 8

#### Read the ground truth file


In [ ]:
!pip install Bio
from Bio import SeqIO

     |████████████████████████████████| 71kB 3.0MB/s 
     |████████████████████████████████| 2.3MB 8.6MB/s 


Find the proteins containing the target domain (PF08543) in:
- SwissProt (annotated) 
- TrEMBLE (not annotated) ?????

Find the proteins containing the target domain (PF08543) in SwissProt -> set of actual positives 


In [ ]:
# !pip install Bio
# from Bio import SeqIO
path = datapath + 'protein-matching-PF08543.fasta'
protein_matching = SeqIO.parse(path,'fasta')

list_protein_containing_domain = []
for sequence in protein_matching:
    # print(sequence)
    name = sequence.id # name is in the form A1ADT5|reviewed|Pyridoxine/pyridoxal/pyridoxamine
    # print('name ->', name, '\n')
    list_protein_containing_domain.append(name.split('|')[0])

set_protein_containing_domain = set(list_protein_containing_domain) # this is the ground truth
print(f"Number of proteins in SwissProt containing the target domain: {len(list_protein_containing_domain)}")
# this is the total number of protein in SwissProt
print(f"Actual positives: {set_protein_containing_domain}")

Number of proteins in SwissProt containing the target domain: 150
Actual positives: {'Q2L1P5', 'Q7VYK4', 'Q8Z6Q3', 'Q9USL6', 'O74860', 'Q5PIK8', 'Q87TZ6', 'Q1AYE5', 'B1JFM7', 'P44697', 'Q6NG19', 'A7ZPL9', 'Q9ZL00', 'Q62LP6', 'P61422', 'Q9ZBL1', 'Q1CIM6', 'Q5PNC8', 'B7LCG3', 'B6I4Z5', 'P76422', 'B7N609', 'Q8Z4W1', 'Q8FFB5', 'A7N5Q6', 'Q63SC2', 'P56904', 'A3N2D3', 'A8A2R4', 'Q0TF48', 'Q2SXQ4', 'Q8W1X2', 'C3K4G7', 'Q9CNY1', 'Q320Z3', 'Q6AFC1', 'Q1LFU5', 'B2JCI0', 'Q83K78', 'Q8FH89', 'B5YZW5', 'Q141E8', 'P66915', 'A5WB73', 'Q32FD7', 'Q6GEY2', 'Q3YZC3', 'B7UGB9', 'Q7W6K7', 'Q65UE8', 'O31620', 'Q4K3F6', 'O67772', 'B7M6S8', 'O14242', 'Q8K183', 'Q52975', 'Q7MGA4', 'Q51892', 'B0UUD2', 'Q6D5V1', 'A6VEZ4', 'Q3Z1Z2', 'Q1BXQ7', 'Q0BSF0', 'B7V753', 'Q3K4B8', 'Q66A50', 'Q7CIR8', 'Q55EK9', 'Q5HRG7', 'B2TX08', 'O85786', 'Q2QWK9', 'Q57LS3', 'P39988', 'Q0II59', 'Q5HI96', 'B7NPV5', 'Q08224', 'B7MHS2', 'C4ZVU9', 'B1XA89', 'Q06490', 'Q48BL6', 'Q8XBL0', 'A6VNE5', 'O00764', 'Q3JQA6', 'B7MY71', 'Q8CTQ7', 'Q32D

Find all the proteins in the target database (uniprot_sprot.fasta) -> complete ground truth (actual positives + actual negatives)

In [ ]:
path = datapath + 'uniprot_sprot.fasta'
all_sequences = SeqIO.parse(path, 'fasta')

list_all_sequences = []
for sequence in all_sequences:
    name = sequence.id # name is in the form sp|Q6GZX4|001R_FRG3G
    list_all_sequences.append(name.split('|')[1])

num_sequences = len(set(list_all_sequences))
print(f"Total number of proteins in SwissProt: {num_sequences}")

Total number of proteins in SwissProt: 563972


#### Read the output file of the psi-blast search

In [ ]:
def read_psiblast(path):
    ''' Create a dictionary having as key the protein ID and as value the set of
    tuples (start, end) representing the position of our domain in that protein
    (we assume that the domain may be present in more than one position). '''
    
    positions = {}
    with open(path, 'r') as f:
        for line in f:
            if line:
                line = line.replace('|', ' ')
                (_, _, id, _, _, length, _, gapopen, _, _, 
                    start, end, evalue, bitscore) = line.split()
                    
                positions.setdefault(id, set()).add((start, end)) 
    return positions

In [ ]:
path = datapath + 'psiblast_search_uniref50.txt'
positions_psiblast = read_psiblast(path)
print(f'Number of distint proteins found with psi-blast: {len(positions_psiblast)}')

Number of distint proteins found with psi-blast: 711


In [ ]:
positions_psiblast

{'A0A0H2WZY4': {('106', '296')},
 'A0AFC3': {('15', '249'), ('20', '242'), ('21', '229')},
 'A0KPL4': {('173', '290'), ('83', '317')},
 'A0KTG8': {('129', '290'), ('85', '295')},
 'A0LP26': {('12', '250'), ('19', '243'), ('49', '243')},
 'A0Q299': {('23', '253'), ('8', '249'), ('83', '230')},
 'A0R980': {('13', '252'), ('24', '252')},
 'A1A6H3': {('173', '373'), ('229', '373')},
 'A1ACX0': {('22', '255'), ('25', '255'), ('57', '255')},
 'A1ADT5': {('14', '269'), ('14', '278'), ('18', '278')},
 'A1AFX4': {('124', '290'), ('124', '295'), ('124', '312')},
 'A1AV12': {('134', '299'), ('199', '298')},
 'A1JQJ2': {('18', '256'), ('24', '254'), ('47', '246'), ('51', '246')},
 'A1JQV6': {('122', '290'), ('124', '295')},
 'A1RGD1': {('135', '290'), ('85', '295')},
 'A1S0R2': {('321', '476')},
 'A1SX95': {('12', '257')},
 'A1SZY2': {('124', '291'), ('124', '296')},
 'A1TYR4': {('191', '285')},
 'A1VBV4': {('10', '254')},
 'A2BLC0': {('305', '474')},
 'A2RGF2': {('118', '277')},
 'A2RKJ5': {('18'

#### Read the output file of the hmm search

In [ ]:
def read_hmm(path):

    ''' Create a dictionary having as key the protein ID and as value the set of
    tuples (start, end) representing the position of our domain in that protein
    (we assume that the domain may be present in more than one position). '''
  
    positions = {}
    with open(path,'r') as f:
        for line in f:
            if line[0] != "#": # and line[0] != "-":
              line=line.replace('|', ' ')
              (_, id, _, _, tlen, _, _, qlen,  
              full_seq_evalue, full_seq_score, _, _, _, dcevalue, dievalue, dscore, _, _, _ , 
              align_from, align_to, env_from, env_to, accuracy, description) = line.strip().split()[:25]
              #exist multiple scores and alignment. Above I took all of them, but in the below dictionary
              #I have put the same used in github reference
            
              positions.setdefault(id, set()).add( (align_from, align_to) )
   
    return positions


In [ ]:
path = datapath + 'hmm_search_uniref50.hmmer_domtblout'
positions_hmm = read_hmm(path)
print(f'Number of distint proteins found with hmm: {len(positions_hmm)}')

Number of distint proteins found with hmm: 723


In [ ]:
positions_hmm

{'A0A0H2WZY4': {('163', '223')},
 'A0AFC5': {('22', '205')},
 'A0JYP1': {('24', '230')},
 'A0KPL4': {('189', '289')},
 'A0KTG8': {('189', '289')},
 'A0LP26': {('79', '219')},
 'A0LTK3': {('28', '205')},
 'A0PRY2': {('24', '214')},
 'A0Q2A0': {('5', '201')},
 'A0QLT6': {('25', '219')},
 'A0QQK7': {('29', '223')},
 'A0R980': {('170', '241')},
 'A0R981': {('25', '211')},
 'A0RQR9': {('183', '216')},
 'A1A6H3': {('243', '349')},
 'A1ADT5': {('89', '268')},
 'A1AFX4': {('189', '287')},
 'A1AIG4': {('19', '202')},
 'A1AV12': {('199', '333')},
 'A1BGM7': {('25', '199')},
 'A1JQJ2': {('70', '223')},
 'A1JQV6': {('189', '289')},
 'A1KB60': {('16', '198')},
 'A1KFN6': {('23', '218')},
 'A1RGD1': {('189', '289')},
 'A1S0R2': {('376', '469')},
 'A1S3I3': {('189', '288')},
 'A1SZY2': {('190', '290')},
 'A1T2Z5': {('24', '213')},
 'A1TYG6': {('22', '204')},
 'A1TYR4': {('188', '324')},
 'A1UAB4': {('29', '223')},
 'A1W068': {('21', '203')},
 'A1WYL4': {('19', '203')},
 'A2BSJ5': {('151', '340')},
 '

#### Model evaluation

In [ ]:
# !pip install Bio
# from Bio import SeqIO
import math
import json
import pandas as pd
import numpy as np

#### Useful functions to compute metrics

CONFIDENTIAL!!! :-)

In $\verb!utils.py!$ there are 5 functions:

- $\verb!computeMetrics!$
- $\verb!createPositionSet!$
- $\verb!evaluateSequencesSH2!$
- $\verb!getSeqLength!$
- $\verb!evaluatePositionsSH2!$

But for point 8) are needed only:
- $\verb!computeMetrics!$
- $\verb!evaluateSequencesSH2!$

So I use only them in the following.

In [ ]:
# import math
def metrics(true_positive, true_negative, false_positive, false_negative):
  
    accuracy = (true_positive + true_negative) / (true_positive + true_negative + false_negative + false_positive)
    precision = true_positive / (true_positive + false_positive)
    sensitivity = true_positive / (true_positive + false_negative)
    specificity = true_negative / (true_negative + false_positive)
    weighted_accuracy = (sensitivity + specificity) / 2
    mcc_numerator = (true_positive * true_negative) - (false_positive * false_negative)
    mcc_denominator_squared = (true_positive + false_positive) * (true_positive + false_negative) * (true_negative + false_positive) * (true_negative + false_negative)
    mcc = mcc_numerator / math.sqrt(mcc_denominator_squared)
    f1_score = 2 * (precision * sensitivity) / (precision + sensitivity)
    
    return {
            "Accuracy": accuracy,
            "Weighted accuracy": weighted_accuracy,
            "Precision": precision,
            "Sensitivity": sensitivity,
            "Specificity": specificity,
            "MCC": mcc,
            "F-score": f1_score
            }


In [ ]:
def metrics_results(predictions, actual_positives, ground_truth_size):
    """
    predictions = dict of the type {'A0A0H2WZY4': {('167', '228')}, ... }
    actual_positives = set of sequences containing PF08543
    ground_truth_size = number of actual positives + actual negatives (= SwissPort size)
    """
    predicted_sequences = set(predictions.keys())
    
    true_positive = len(predicted_sequences & actual_positives)
    
    # # If there aren't true postive later there will be a division by 0
    # if true_positive == 0:
    #     print("No true positive!")
    #     return
    
    false_negative = len(actual_positives) - true_positive
    false_positive = len(predicted_sequences) - true_positive
    true_negative = ground_truth_size - true_positive - false_positive - false_negative
    
    conf_matrix = pd.DataFrame(np.array([[true_positive, false_positive], [false_negative, true_negative]]), 
                               index=['Positive pred', 'Negative pred'], 
                               columns=['Actual positive', 'Actual negative'])

    return conf_matrix, metrics(true_positive, true_negative, false_positive, false_negative)  



#### Evaluate psi-blast

In [ ]:
conf_matrix_psiblast, metric_results_psiblast = metrics_results(positions_psiblast, set_protein_containing_domain, num_sequences )
print('CLASSIFICATION METRICS')
for k in metric_results_psiblast:
    print(f'{k}: {metric_results_psiblast[k]:.3}')
    
print('\nCONFUSION MATRIX')    
conf_matrix_psiblast

CLASSIFICATION METRICS
Accuracy: 0.999
Weighted accuracy: 0.996
Precision: 0.21
Sensitivity: 0.993
Specificity: 0.999
MCC: 0.456
F-score: 0.346

CONFUSION MATRIX


,Actual positive,Actual negative
Positive pred,149,562
Negative pred,1,563260


#### Evaluate hmm

In [ ]:
conf_matrix_hmm, metric_results_hmm = metrics_results(positions_hmm, set_protein_containing_domain, num_sequences )
print('CLASSIFICATION METRICS')
for k in metric_results_hmm:
    print(f'{k}: {metric_results_hmm[k]:.3}')
    
print('\nCONFUSION MATRIX')    
conf_matrix_hmm

CLASSIFICATION METRICS
Accuracy: 0.999
Weighted accuracy: 0.999
Precision: 0.207
Sensitivity: 1.0
Specificity: 0.999
MCC: 0.455
F-score: 0.344

CONFUSION MATRIX


,Actual positive,Actual negative
Positive pred,150,573
Negative pred,0,563249


### Comment on the results

We have so many false positives and very few false negatives. Probably we should try to retrieve less hits, since their correct number is 150, but our models selected much more sequences (674 with psiblast and 412 with hmm).

## Ex 9

#### Read the ground truth file for the positions

In [ ]:
path = datapath + 'protein-matching-PF08543.json'

with open(path, "r") as file:
    position_file = json.load(file)
    
ground_truth_positions = {}
for elem in position_file:
    id = elem['metadata']['accession']
    length = elem['metadata']['length']

    ground_truth_positions.setdefault(id, dict()).update([('length', length), ('positions', set())])

    for entry in elem['entries']:
        if entry['accession']=='PF08543':
            for location in entry['entry_protein_locations']:

                position_tuple=(str(location['fragments'][0]['start']),
                                str(location['fragments'][0]['end']))

                ground_truth_positions[id]['positions'].add(position_tuple)

In [ ]:
print(len(ground_truth_positions))

150


In [ ]:
for item in ground_truth_positions.items():
  print(item)

('A1ADT5', {'length': 283, 'positions': {('88', '265')}})
('A3N2D3', {'length': 286, 'positions': {('64', '256')}})
('A5UA83', {'length': 288, 'positions': {('71', '264')}})
('A5WB73', {'length': 290, 'positions': {('74', '257')}})
('A6VEZ4', {'length': 288, 'positions': {('77', '259')}})
('A6VNE5', {'length': 286, 'positions': {('62', '259')}})
('A7N5Q6', {'length': 289, 'positions': {('75', '260')}})
('A7ZPL9', {'length': 283, 'positions': {('88', '265')}})
('A8A2R4', {'length': 283, 'positions': {('88', '265')}})
('B0KR83', {'length': 290, 'positions': {('76', '257')}})
('B0UUD2', {'length': 286, 'positions': {('64', '260')}})
('B1IX53', {'length': 283, 'positions': {('88', '265')}})
('B1JFM7', {'length': 290, 'positions': {('76', '257')}})
('B1LML3', {'length': 283, 'positions': {('88', '265')}})
('B1XA89', {'length': 283, 'positions': {('88', '265')}})
('B2JCI0', {'length': 287, 'positions': {('74', '261')}})
('B2TX08', {'length': 283, 'positions': {('88', '265')}})
('B3H2H2', {'l

#### Useful functions to compute metrics

CONFIDENTIAL!!! :-)

In $\verb!utils.py!$ there are 5 functions:

- $\verb!computeMetrics!$
- $\verb!createPositionSet!$
- $\verb!evaluateSequencesSH2!$
- $\verb!getSeqLength!$
- $\verb!evaluatePositionsSH2!$

But for point 9) are needed only:
- $\verb!computeMetrics!$
- $\verb!createPositionSet!$
- $\verb!getSeqLength!$
- $\verb!evaluatePositionsSH2!$

So I use only them in the following.

In [ ]:
def createPositionSet(position_set):
    """
    position_set = {('88', '265'), ...}
    return a set {88, 89, ..., 265, ...}
    """
    positions = set()
    for pos in position_set: # pos is a tuple of two strings
        p = [i for i in range( int(pos[0]), int(pos[1])+1 )]
        positions = positions.union(p)
    return positions

This is the original function.

I'm not able to run it because it calls a command line in Linux. 

If you can, please try to see if the below function provides the same result as my version.

I get the following error...

`Error: [blastdbcmd] DB contains no accession info.`

In [ ]:
############### ORIGINAL FUNCTION
'''
def getSeqLength(entry):
    """
    Get sequence length from reference dataset
    """   
    cmd = 'blastdbcmd -entry "{}" -db ../../data/SwissProt_humans_reference_all.fasta'.format(entry)
    results = subprocess.run(
            cmd, shell=True, universal_newlines=True, 
            stdout=subprocess.PIPE
            ).stdout
    results = results.split('\n')
    results.pop(0)
    return len("".join(results))
'''

'\ndef getSeqLength(entry):\n    """\n    Get sequence length from reference dataset\n    """   \n    cmd = \'blastdbcmd -entry "{}" -db ../../data/SwissProt_humans_reference_all.fasta\'.format(entry)\n    results = subprocess.run(\n            cmd, shell=True, universal_newlines=True, \n            stdout=subprocess.PIPE\n            ).stdout\n    results = results.split(\'\n\')\n    results.pop(0)\n    return len("".join(results))\n'

In [ ]:
path = datapath + 'uniprot_sprot.fasta'
all_sequences = SeqIO.parse(path, 'fasta')

In [ ]:
##################### MY VERSION
def getSeqLength(entry, all_sequences):

  """
  Get sequence length from reference dataset
  """ 
#   path = datapath + 'uniprot_sprot.fasta'
#   all_sequences = SeqIO.parse(path, 'fasta')

  list_all_sequences = []
  for sequence in all_sequences:
      name = sequence.id # name is in the form sp|Q6GZX4|001R_FRG3G
      name = name.split('|')[1]
      if name==entry:
        #print(sequence.seq)
        #print(len(sequence.seq))
        length=len(sequence.seq)
  return length


In [ ]:
#-------------------------DEBUG
entry='Q6GZX4'
getSeqLength(entry, all_sequences)

256

In [ ]:
def evaluatePositions(predicted_positions, ground_truth_positions):
    """
    predicted_positions: {'A0A0H2WZY4': {('162', '278')}, ...}
    ground_truth_positions: {'A1ADT5': {'length': 283, 'positions': {('88', '265')}}, ...}
    
    """
    global_tp, global_fp, global_fn, global_tn = [], [], [], []
    
    # scores per sequence
    log_scores = []

    for id in predicted_positions:
        if id in ground_truth_positions:
            sequence_length = ground_truth_positions[id]['length']
            ground_truth_positions_seq = createPositionSet(ground_truth_positions[id]['positions']) 
            ####################################################################################
            # Here there was an error: there was reference_positions = createPositionSet(reference_positions[seqid]['positions']), 
            # which modifies reference_positions! So we had correct results only for the first protein!
            ####################################################################################
            predicted_positions_seq = createPositionSet(predicted_positions[id])    

            overlapping_positions  = ground_truth_positions_seq & predicted_positions_seq

            true_positive = len(overlapping_positions)
            false_positive = len(predicted_positions_seq - overlapping_positions)
            false_negative = len(ground_truth_positions_seq -  overlapping_positions)
            true_negative  = sequence_length - true_positive - false_positive - false_negative

            global_tp.append(true_positive)
            global_fp.append(false_positive)
            global_fn.append(false_negative)
            global_tn.append(true_negative)
            
            log_scores.append(metrics(true_positive, true_negative, false_positive, false_negative))


        ##############
        # HA SENSO? ci mette una vita per calcolare la lunghezza delle sequence con getSeqLength 
        # e non so se sia utile/sensato comprendere le sequenze senza dominio in questo esercizio
        ##############
        # else:
        #     # false positive sequence
        #     predicted_positions_seq = createPositionSet(predicted_positions[id])

        #     true_positive = 0
        #     false_positive = len(predicted_positions_seq)
        #     false_negative = 0
        #     true_negative = 0
            
        #     lenSeq = getSeqLength(id)
            
        #     log_scores.append(
        #         {
        #             'Precision': 0,
        #             'Sensitivity': 0,
        #             'Specificity': lenSeq / (lenSeq + false_positive),
        #             'F1-score': 0,
        #             'MCC': 0
        #         }
        #     )

        #     list_tp.append(true_positive)
        #     list_fp.append(false_positive)
        #     list_fn.append(false_negative)
        #     list_tn.append(true_negative)
                   
    metrics_global = metrics(true_positive=sum(global_tp), true_negative=sum(global_tn), 
                             false_positive=sum(global_fp), false_negative=sum(global_fn))
    
    conf_matrix = pd.DataFrame(np.array([[sum(global_tp), sum(global_fp)], [sum(global_fn), sum(global_tn)]]), 
                               index=['Positive pred', 'Negative pred'], 
                               columns=['Actual positive', 'Actual negative'])
    
    return metrics_global, log_scores, conf_matrix

#### Evaluate psi-blast

In [ ]:
metrics_global_psiblast, log_score_psiblast, conf_matrix_psiblast_pos = evaluatePositions(positions_psiblast, ground_truth_positions)

ZeroDivisionError: ignored

In [ ]:
print('CLASSIFICATION METRICS')
for k in metrics_global_psiblast:
    print(f'{k}: {metrics_global_psiblast[k]:.3}')
print('\nCONFUSION MATRIX')    
conf_matrix_psiblast_pos

CLASSIFICATION METRICS


NameError: ignored

In [ ]:
# This is the output considering also the retrieved sequences without the domain
metrics_global_psiblast

{'Accuracy': 0.2367829747669103,
 'F1-score': 0.3334987313222441,
 'MCC': 0.10470260431297877,
 'Precision': 0.20015431367638797,
 'Sensitivity': 0.9991215919456738,
 'Specificity': 0.05666642719733071}

In [ ]:
# old version
metrics_global_psiblast

{'Accuracy': 0.0033580173508824087,
 'F1-score': 0.003297408696239738,
 'MCC': 0.0016815567089388617,
 'Precision': 0.0016514270630994038,
 'Sensitivity': 1.0,
 'Specificity': 0.0017122360584731998}

In [ ]:
log_score_psiblast

[{'Accuracy': 0.9521072796934866,
  'F-score': 0.9512670565302145,
  'MCC': 0.9085663242927233,
  'Precision': 0.9070631970260223,
  'Sensitivity': 1.0,
  'Specificity': 0.9100719424460432,
  'Weighted accuracy': 0.9550359712230216},
 {'Accuracy': 0.9582577132486388,
  'F-score': 0.957169459962756,
  'MCC': 0.9198108458946662,
  'Precision': 0.9178571428571428,
  'Sensitivity': 1.0,
  'Specificity': 0.9217687074829932,
  'Weighted accuracy': 0.9608843537414966},
 {'Accuracy': 0.9398496240601504,
  'F-score': 0.96875,
  'MCC': 0.32307445639837323,
  'Precision': 0.9393939393939394,
  'Sensitivity': 1.0,
  'Specificity': 0.1111111111111111,
  'Weighted accuracy': 0.5555555555555556},
 {'Accuracy': 0.9714285714285714,
  'F-score': 0.9722222222222222,
  'MCC': 0.944400281603035,
  'Precision': 0.9459459459459459,
  'Sensitivity': 1.0,
  'Specificity': 0.9428571428571428,
  'Weighted accuracy': 0.9714285714285714},
 {'Accuracy': 0.9511278195488722,
  'F-score': 0.9744597249508841,
  'MCC': 

In [ ]:
# old version
score_sequence_psiblast

[{'Accuracy': 0.9521072796934866,
  'F1-score': 0.9512670565302145,
  'MCC': 0.9085663242927233,
  'Precision': 0.9070631970260223,
  'Sensitivity': 1.0,
  'Specificity': 0.9100719424460432},
 {'F1-score': 0,
  'MCC': 0,
  'Precision': 0,
  'Sensitivity': 0,
  'Specificity': 0.6630565583634176},
 {'F1-score': 0,
  'MCC': 0,
  'Precision': 0,
  'Sensitivity': 0,
  'Specificity': 0.5018867924528302},
 {'F1-score': 0,
  'MCC': 0,
  'Precision': 0,
  'Sensitivity': 0,
  'Specificity': 0.6542056074766355},
 {'F1-score': 0,
  'MCC': 0,
  'Precision': 0,
  'Sensitivity': 0,
  'Specificity': 0.5047438330170778},
 {'F1-score': 0,
  'MCC': 0,
  'Precision': 0,
  'Sensitivity': 0,
  'Specificity': 0.661188369152971},
 {'F1-score': 0,
  'MCC': 0,
  'Precision': 0,
  'Sensitivity': 0,
  'Specificity': 0.5092250922509225},
 {'F1-score': 0,
  'MCC': 0,
  'Precision': 0,
  'Sensitivity': 0,
  'Specificity': 0.5092250922509225},
 {'F1-score': 0,
  'MCC': 0,
  'Precision': 0,
  'Sensitivity': 0,
  'Spec

#### Evaluate hmm

In [ ]:
metrics_global_hmm, log_score_hmm, conf_matrix_hmm_pos = evaluatePositions(positions_hmm, ground_truth_positions)

In [ ]:
print('CLASSIFICATION METRICS')
for k in metrics_global_hmm:
    print(f'{k}: {metrics_global_hmm[k]:.3}')
print('\nCONFUSION MATRIX')    
conf_matrix_hmm_pos

CLASSIFICATION METRICS
Accuracy: 0.88
Weighted accuracy: 0.865
Precision: 0.898
Sensitivity: 0.916
Specificity: 0.814
MCC: 0.737
F-score: 0.907

CONFUSION MATRIX


,Actual positive,Actual negative
Positive pred,27121,3066
Negative pred,2478,13446


In [ ]:
metrics_global_hmm

{'Accuracy': 0.8797683849840602,
 'F-score': 0.9072692603619577,
 'MCC': 0.7366830323110901,
 'Precision': 0.8984331003412065,
 'Sensitivity': 0.9162809554376837,
 'Specificity': 0.8143168604651163,
 'Weighted accuracy': 0.8652989079514}

In [ ]:
log_score_hmm

[{'Accuracy': 0.9734693877551021,
  'F-score': 0.9737373737373737,
  'MCC': 0.9471360338761622,
  'Precision': 0.964,
  'Sensitivity': 0.9836734693877551,
  'Specificity': 0.963265306122449,
  'Weighted accuracy': 0.9734693877551021},
 {'Accuracy': 0.9726277372262774,
  'F-score': 0.9696969696969697,
  'MCC': 0.9449035066900258,
  'Precision': 0.96,
  'Sensitivity': 0.9795918367346939,
  'Specificity': 0.966996699669967,
  'Weighted accuracy': 0.9732942682023304},
 {'Accuracy': 0.9398496240601504,
  'F-score': 0.967871485943775,
  'MCC': 0.5017285475203439,
  'Precision': 0.9601593625498008,
  'Sensitivity': 0.9757085020242915,
  'Specificity': 0.47368421052631576,
  'Weighted accuracy': 0.7246963562753036},
 {'Accuracy': 0.9516728624535316,
  'F-score': 0.9741550695825051,
  'MCC': 0.6104374621209737,
  'Precision': 0.9645669291338582,
  'Sensitivity': 0.9839357429718876,
  'Specificity': 0.55,
  'Weighted accuracy': 0.7669678714859438},
 {'Accuracy': 0.9712643678160919,
  'F-score': 

In [ ]:
score_sequence_hmm

[{'Accuracy': 0.9734693877551021,
  'F1-score': 0.9737373737373737,
  'MCC': 0.9471360338761622,
  'Precision': 0.964,
  'Sensitivity': 0.9836734693877551,
  'Specificity': 0.963265306122449},
 {'F1-score': 0,
  'MCC': 0,
  'Precision': 0,
  'Sensitivity': 0,
  'Specificity': 0.6867167919799498},
 {'F1-score': 0,
  'MCC': 0,
  'Precision': 0,
  'Sensitivity': 0,
  'Specificity': 0.5145067698259188},
 {'F1-score': 0,
  'MCC': 0,
  'Precision': 0,
  'Sensitivity': 0,
  'Specificity': 0.51434034416826},
 {'F1-score': 0,
  'MCC': 0,
  'Precision': 0,
  'Sensitivity': 0,
  'Specificity': 0.6752910737386805},
 {'F1-score': 0,
  'MCC': 0,
  'Precision': 0,
  'Sensitivity': 0,
  'Specificity': 0.516504854368932},
 {'F1-score': 0,
  'MCC': 0,
  'Precision': 0,
  'Sensitivity': 0,
  'Specificity': 0.5213178294573644},
 {'F1-score': 0,
  'MCC': 0,
  'Precision': 0,
  'Sensitivity': 0,
  'Specificity': 0.5155038759689923},
 {'F1-score': 0,
  'MCC': 0,
  'Precision': 0,
  'Sensitivity': 0,
  'Speci